In [5]:
import CuponCero as cc
import pandas as pd 

| Codigo | Descripciòn |
| ------ | ------ |
| CBCRS |    Curva Cupon Cero CD BCRP|
| CCSDF |   Curva Cupon Cero Dólares CP|
| CSBCRD | Curva Cupon Cero Dólares Sintetica|
| CCINFS | Curva Cupon Cero Inflacion Soles BCRP|
| CCCLD | Curva Cupon Cero Libor Dolares|
| CCPEDS | Curva Cupon Cero Peru Exterior Dolares - Soberana|
| CCPSS | Curva Cupon Cero Peru Soles Soberana|
| CCPVS | Curva Cupon Cero Peru Vac Soberana|

In [6]:

tp_curva = 'CCPSS'
fec_proceso = '31/07/2023'
df_cup= cc.get_curva_cupon_cero(tipoCurva=tp_curva, fechaProceso=fec_proceso)



In [7]:
df_cup['Tasas (%)'] = pd.to_numeric(df_cup['Tasas (%)'], errors='coerce')
df_cup['Periodo (días)'] = pd.to_numeric(df_cup['Periodo (días)'], errors='coerce')

In [16]:
df_cup.head(20)

,Sec.,Fecha de Proceso,Periodo (días),Tasas (%)
0,1,31/07/2023,0,7.223151
1,2,31/07/2023,90,6.541085
2,3,31/07/2023,180,6.135739
3,4,31/07/2023,270,5.910281
4,5,31/07/2023,360,5.800567
5,6,31/07/2023,450,5.764073
6,7,31/07/2023,540,5.772694
7,8,31/07/2023,630,5.807966
8,9,31/07/2023,720,5.857879
9,10,31/07/2023,810,5.914737


In [9]:
df_cup_por_anio = df_cup[df_cup['Periodo (días)'] % 360 == 0].copy()

df_cup_por_anio["anio"] = df_cup_por_anio['Periodo (días)'] / 360 


In [10]:
df_cup_por_anio.head(10)

,Sec.,Fecha de Proceso,Periodo (días),Tasas (%),anio
0,1,31/07/2023,0,7.223151,0.0
4,5,31/07/2023,360,5.800567,1.0
8,9,31/07/2023,720,5.857879,2.0
12,13,31/07/2023,1080,6.087532,3.0
16,17,31/07/2023,1440,6.273800,4.0
20,21,31/07/2023,1800,6.404964,5.0
24,25,31/07/2023,2160,6.497120,6.0
28,29,31/07/2023,2520,6.563931,7.0
32,33,31/07/2023,2880,6.614167,8.0
36,37,31/07/2023,3240,6.653211,9.0


In [11]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=df_cup_por_anio.anio, y=df_cup_por_anio["Tasas (%)"], mode='lines+markers',    name='lines+markers'))

# Obtener los límites mínimo y máximo de los datos en el eje Y
y_min = df_cup_por_anio["Tasas (%)"].min()
y_max = df_cup_por_anio["Tasas (%)"].max()

# Calcular los 8 valores equidistantes entre el límite mínimo y máximo
y_values = [y_min + (i * (y_max - y_min) / 7) for i in range(8)]

fig.update_layout(
                   xaxis_title='Años',
                   yaxis_title='Tasas',       
                   yaxis=dict(tickmode='array',  tickvals=y_values, nticks=8, tickfont=dict(size=12), hoverformat='.2f'),         
                   xaxis=dict(type='category', tickfont=dict(size=12), tickangle=90),
                   margin=dict(l=20, r=10, t=20, b=10)
                   )

fig.show()

In [21]:
import pandas as pd
import numpy as np

# DataFrame A con los datos proporcionados
data_test = {
    "días": [0, 30, 60, 90, 120],    
}
df_test = pd.DataFrame(data_test)


def get_pronostico_lineal(conjunto_x,conjunto_y, var_indep ):
    x = var_indep
    f = np.polyfit(conjunto_x, conjunto_y, 1)
    a = f[0]
    b = f[1]
    pronostico = a * x + b

    return pronostico


def get_tasa_interes_por_dias(dias ,df_tasas):

    lb_dias = "Periodo (días)"
    ld_tasas = "Tasas (%)"

    # Valor "x" que deseas buscar
    x = dias
    y = None
    # Si "x" coincide con uno de los valores en la columna "días", entonces se muestra el registro correspondiente
    matching_record = df_tasas[df_tasas[lb_dias] == x]
    if len(matching_record)>0:

        #print(matching_record)
        y = matching_record.loc[: , ld_tasas].values[0]

    else:        
        # Encontrar el registro inferior y el registro superior más próximos a "x" en la columna "días"
        lower_record = df_tasas[df_tasas[lb_dias] <= x].tail(1)
        upper_record = df_tasas[df_tasas[lb_dias] >= x].head(1)

        result = pd.concat([lower_record, upper_record])

        conjunto_x = result[lb_dias]
        conjunto_y = result[ld_tasas]
        
        y = get_pronostico_lineal(conjunto_x,conjunto_y,x)

        #print(result)
    return y        
    #print(y)
    # Combinar los registros encontrados


In [22]:

# Aplicar la función multiplicar_por_valor utilizando apply y pasar el valor_fijo como argumento

df_test['tasas'] = df_test['días'].apply(get_tasa_interes_por_dias, args=(df_cup,))

In [23]:
df_test

,días,tasas
0,0,7.223151
1,30,6.995795
2,60,6.768440
3,90,6.541085
4,120,6.405970
